In [13]:
%%capture
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Conv3D, DepthwiseConv2D, SeparableConv2D, Conv3DTranspose
from keras.layers import Flatten, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D, BatchNormalization
from keras.layers import Concatenate, Add, Dropout, ReLU, Lambda, Activation, LeakyReLU, PReLU
from keras.utils import np_utils
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.optimizers import SGD
from time import time
from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from keras.callbacks import Callback

import numpy as np
import pandas as pd
import matplotlib, os, math

os.environ["CUDA_VISIBLE_DEVICES"]="2"

### 1.Prepare data

In [2]:
from time import time
import numpy as np

start_time = time()

print('Loading X_train...')
X_train = np.load('/home/zxt/data/final_22/224/2k/X_train.npy')


end_time = time()
print('Running time: %s seconds'%(end_time - start_time))

Loading X_train...
Running time: 338.64334440231323 seconds


In [3]:
len(X_train)

35200

In [4]:
from time import time
import numpy as np

start_time = time()

print('Loading X_validate...')
X_validate = np.load('/home/zxt/data/final_22/224/2k/X_validate.npy')

print('Loading X_test...')
X_test = np.load('/home/zxt/data/final_22/224/2k/X_test.npy')

print('Loading labels...')
Y_train = np.load('/home/zxt/data/final_22/224/2k/Y_train.npy')
Y_validate = np.load('/home/zxt/data/final_22/224/2k/Y_validate.npy')
Y_test = np.load('/home/zxt/data/final_22/224/2k/Y_test.npy')

end_time = time()
print('Running time: %s seconds'%(end_time - start_time))

Loading X_validate...
Loading X_test...
Loading labels...
Running time: 85.21720027923584 seconds


In [5]:
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_validate.shape[0], 'validate samples')
print(X_test.shape[0], 'test samples')

X_train shape: (35200, 224, 224, 3)
35200 train samples
4400 validate samples
4400 test samples


In [6]:
X_train /=255.
X_validate /=  255.
X_test /= 255.

In [7]:
Y_train = np.load('/home/zxt/data/final_22/224/2k/Y_train.npy')
Y_validate = np.load('/home/zxt/data/final_22/224/2k/Y_validate.npy')
Y_test = np.load('/home/zxt/data/final_22/224/2k/Y_test.npy')

In [8]:
Y_test

array([['guineafowl'],
       ['lion'],
       ['wildebeest'],
       ...,
       ['buffalo'],
       ['hare'],
       ['black_bear']], dtype='<U16')

In [10]:
# Y_train label
for i in Y_train:
    if i[0]=='elephant':
        i[0]=0
    if i[0]=='wild_boar':
        i[0]=1
    if i[0]=='gazellethomosons':
        i[0]=2
    if i[0]=='squirrel':
        i[0]=3 
    if i[0]=='hedgehog':
        i[0]=4
    if i[0]=='guineafowl':
        i[0]=5
    if i[0]=='moose':
        i[0]=6
    if i[0]=='coyote':
        i[0]=7   
    if i[0]=='bird':
        i[0]=8
    if i[0]=='fox':
        i[0]=9
    if i[0]=='giraffe':
        i[0]=10
    if i[0]=='buffalo':
        i[0]=11
    if i[0]=='hare':
        i[0]=12
    if i[0]=='vehicle':
        i[0]=13
    if i[0]=='wildebeest':
        i[0]=14
    if i[0]=='cattle':
        i[0]=15 
    if i[0]=='skunk':
        i[0]=16
    if i[0]=='lion':
        i[0]=17
    if i[0]=='zebra':
        i[0]=18
    if i[0]=='black_bear':
        i[0]=19
    if i[0]=='racoon':
        i[0]=20
    if i[0]=='empty':
        i[0]=21
        
for i in Y_validate:
    if i[0]=='elephant':
        i[0]=0
    if i[0]=='wild_boar':
        i[0]=1
    if i[0]=='gazellethomosons':
        i[0]=2
    if i[0]=='squirrel':
        i[0]=3 
    if i[0]=='hedgehog':
        i[0]=4
    if i[0]=='guineafowl':
        i[0]=5
    if i[0]=='moose':
        i[0]=6
    if i[0]=='coyote':
        i[0]=7   
    if i[0]=='bird':
        i[0]=8
    if i[0]=='fox':
        i[0]=9
    if i[0]=='giraffe':
        i[0]=10
    if i[0]=='buffalo':
        i[0]=11
    if i[0]=='hare':
        i[0]=12
    if i[0]=='vehicle':
        i[0]=13
    if i[0]=='wildebeest':
        i[0]=14
    if i[0]=='cattle':
        i[0]=15 
    if i[0]=='skunk':
        i[0]=16
    if i[0]=='lion':
        i[0]=17
    if i[0]=='zebra':
        i[0]=18
    if i[0]=='black_bear':
        i[0]=19
    if i[0]=='racoon':
        i[0]=20
    if i[0]=='empty':
        i[0]=21

for i in Y_test:
    if i[0]=='elephant':
        i[0]=0
    if i[0]=='wild_boar':
        i[0]=1
    if i[0]=='gazellethomosons':
        i[0]=2
    if i[0]=='squirrel':
        i[0]=3 
    if i[0]=='hedgehog':
        i[0]=4
    if i[0]=='guineafowl':
        i[0]=5
    if i[0]=='moose':
        i[0]=6
    if i[0]=='coyote':
        i[0]=7   
    if i[0]=='bird':
        i[0]=8
    if i[0]=='fox':
        i[0]=9
    if i[0]=='giraffe':
        i[0]=10
    if i[0]=='buffalo':
        i[0]=11
    if i[0]=='hare':
        i[0]=12
    if i[0]=='vehicle':
        i[0]=13
    if i[0]=='wildebeest':
        i[0]=14
    if i[0]=='cattle':
        i[0]=15 
    if i[0]=='skunk':
        i[0]=16
    if i[0]=='lion':
        i[0]=17
    if i[0]=='zebra':
        i[0]=18
    if i[0]=='black_bear':
        i[0]=19
    if i[0]=='racoon':
        i[0]=20
    if i[0]=='empty':
        i[0]=21

In [14]:
from keras.utils import np_utils
y_train=np_utils.to_categorical(Y_train,22)
y_valid=np_utils.to_categorical(Y_validate,22)
y_test=np_utils.to_categorical(Y_test,22)

In [15]:
y_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

### 2.Model structure

In [16]:
def alexnet(input_shape, n_classes):
    input = Input(input_shape)
  
    # actually batch normalization didn't exist back then
    # they used LRN (Local Response Normalization) for regularization
    x = Conv2D(96, 11, strides=4, padding='same', activation='relu')(input)
    x = BatchNormalization()(x)
    x = MaxPool2D(3, strides=2)(x)
  
    x = Conv2D(256, 5, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(3, strides=2)(x)
  
    x = Conv2D(384, 3, strides=1, padding='same', activation='relu')(x)
  
    x = Conv2D(384, 3, strides=1, padding='same', activation='relu')(x)
  
    x = Conv2D(256, 3, strides=1, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(3, strides=2)(x)
  
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dense(4096, activation='relu')(x)
  
    output = Dense(n_classes, activation='softmax')(x)
  
    model = Model(input, output)
    return model

In [13]:
# prevent overfitting
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

### 3.Modeling

In [17]:
num_train_samples = len(Y_train)
num_val_samples = len(Y_validate)
batch_size = 128
epochs = 30
n_classes = 22
input_shape = 224,224,3

learning_rate = 0.01
decay_rate = learning_rate / epochs
momentum = 0.8
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

In [18]:
K.clear_session()
model = alexnet(input_shape, n_classes)
# model.summary()

In [19]:
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

In [20]:
start_time = time()
print('Training model...')


hist = model.fit(
    X_train, 
    y_train,
    batch_size=batch_size,
    epochs=epochs,
#     callbacks=[early_stopping],
    validation_data = (X_validate,y_valid))

end_time = time()
print('Running time: %s seconds'%(end_time - start_time))

Training model...
Train on 35200 samples, validate on 4400 samples
Epoch 1/30
35200/35200 [==============================] - 167s 5ms/step - loss: 1.9912 - accuracy: 0.3769 - val_loss: 2.5199 - val_accuracy: 0.2198
Epoch 2/30
35200/35200 [==============================] - 163s 5ms/step - loss: 1.2911 - accuracy: 0.5587 - val_loss: 1.5951 - val_accuracy: 0.4652
Epoch 3/30
35200/35200 [==============================] - 163s 5ms/step - loss: 1.0182 - accuracy: 0.6508 - val_loss: 1.0634 - val_accuracy: 0.6416
Epoch 4/30
35200/35200 [==============================] - 164s 5ms/step - loss: 0.8096 - accuracy: 0.7226 - val_loss: 1.0403 - val_accuracy: 0.6564
Epoch 5/30
35200/35200 [==============================] - 164s 5ms/step - loss: 0.6516 - accuracy: 0.7790 - val_loss: 1.0106 - val_accuracy: 0.6548
Epoch 6/30
35200/35200 [==============================] - 163s 5ms/step - loss: 0.5077 - accuracy: 0.8283 - val_loss: 0.8186 - val_accuracy: 0.7255
Epoch 7/30
35200/35200 [=====================

In [30]:
# parallel_model.save('/home/zxt/data/model/alex_basic_5k_224.h5')

In [21]:
model.save('/home/zxt/data/final_22/model/224_2k/alexnet_basic.h5')
model.save_weights('/home/zxt/data/final_22/model/224_2k/alexnet_basic_weights.h5')

### 4.Evaluation

In [22]:
history_df = pd.DataFrame(hist.history)
history_df[['loss', 'val_loss']].plot()
history_df[['accuracy', 'val_accuracy']].plot()

### 5.Prediction on X_test dataset

In [23]:
loss,accuracy = model.evaluate(X_test,y_test)
print('\ntest loss',loss)
print('accuracy',accuracy)

4400/4400 [==============================] - 11s 2ms/step

test loss 0.8010427088629116
accuracy 0.8218181729316711


In [96]:
y_pred = model.predict(X_test,batch_size=1)

In [97]:
y_pred

array([[3.78040698e-12, 7.49338511e-08, 4.27519551e-14, ...,
        1.77665662e-12, 9.99998093e-01, 4.28689750e-10],
       [3.16912468e-10, 8.98658072e-07, 3.50001672e-09, ...,
        3.65600243e-08, 2.25727263e-06, 1.09772023e-03],
       [5.02814543e-12, 1.04580656e-12, 2.09564218e-13, ...,
        3.31777547e-13, 8.65537743e-07, 2.95608054e-08],
       ...,
       [9.45886143e-08, 4.44565492e-04, 1.33693973e-10, ...,
        6.07643713e-08, 1.66907504e-01, 2.22583812e-10],
       [3.03537069e-14, 7.70829797e-01, 8.21182532e-13, ...,
        3.85974629e-12, 7.05436709e-10, 2.80322770e-12],
       [7.79408926e-10, 1.31714675e-08, 1.30883793e-11, ...,
        6.41603506e-11, 2.94592772e-09, 9.98352766e-01]], dtype=float32)

In [98]:
len(y_pred)

4200

### 6.Check each categoryaccuracy

In [42]:
a = list(y_pred)
y_prediction=[]
for i in a:
    j =list(i)
    y_prediction.append(j.index(max(j)))

In [43]:
y_prediction[:5]

[19, 12, 7, 12, 10]

In [44]:
Y_test[:5]

array([['19'],
       ['12'],
       ['7'],
       ['12'],
       ['0']], dtype='<U16')

In [45]:
# array to dataframe
y_real=pd.DataFrame(Y_test,columns=['label'])

In [46]:
y_real.head(2)

,label
0,19
1,12


In [47]:
y_predict = pd.DataFrame(y_prediction,columns=['label_pre'])

In [48]:

y_predict.head(2)

,label_pre
0,19
1,12


In [49]:
prediction_df = pd.concat([y_real,y_predict],axis=1)
prediction_df['label']=prediction_df['label'].astype(int)
prediction_df['diff']=prediction_df['label']-prediction_df['label_pre']


In [50]:
prediction_df

,label,label_pre,diff
0,19,19,0
1,12,12,0
2,7,7,0
3,12,12,0
4,0,10,-10
...,...,...,...
4195,9,9,0
4196,10,10,0
4197,17,17,0
4198,15,1,14


In [51]:
prediction_df.to_csv(r'/home/zxt/data/results/prediction_df.csv',index=False)

In [52]:
onehot_keylist = ['elephant','wild_boar','gazellethomosons','squirrel','hedgehog','guineafowl','moose','coyote','bird',
                  'fox','giraffe','buffalo','hare','vehicle','wildebeest','cattle','skunk','lion','zebra','black_bear',
                  'racoon']
onehot_dict = {'elephant':0,'wild_boar':1,'gazellethomosons':2,'squirrel':3,'hedgehog':4,'guineafowl':5,'moose':6,'coyote':7,
              'bird':8,'fox':9,'giraffe':10,'buffalo':11,'hare':12,'vehicle':13,'wildebeest':14,'cattle':15,'skunk':16,'lion':17,
              'zebra':18,'black_bear':19,'racoon':20}

for animal in onehot_keylist:
    name = animal
    animal = prediction_df[prediction_df['label']== onehot_dict[animal]]
    acc = animal.loc[:,'diff'].value_counts()[0]/len(animal)
    print("%s acc: %10.3f" %(name,acc) )

elephant acc:      0.662
wild_boar acc:      0.692
gazellethomosons acc:      0.905
squirrel acc:      0.889
hedgehog acc:      0.917
guineafowl acc:      0.879
moose acc:      0.968
coyote acc:      0.760
bird acc:      0.892
fox acc:      0.807
giraffe acc:      0.821
buffalo acc:      0.761
hare acc:      0.920
vehicle acc:      0.854
wildebeest acc:      0.695
cattle acc:      0.560
skunk acc:      0.847
lion acc:      0.744
zebra acc:      0.790
black_bear acc:      0.808
racoon acc:      0.982
